In [1]:
import os
from tqdm import tqdm
from qecsim.models.rotatedplanar import RotatedPlanarCode
from src.stim_noise_models import get_noise_model
from src.circuit_builder import build_stim_circuit
import jax.numpy as jnp
from jax import random
import json

key = random.key(0) # Only affects the random deformations and basis not the circuit noise

# Collect data

In [50]:
data_points = 1000
rounds = 20
code_width, code_height = 7, 7
noise_model_name = "spin"

code = RotatedPlanarCode(code_width, code_height)
noise_model = get_noise_model(noise_model_name)
num_syndrome_qubits = code.n_k_d[0] - 1

experiments = []
for _ in tqdm(range(data_points), "Generating data", ncols=200):
    key, deformation_key, basis_key = random.split(key, 3)
    allowed = jnp.array([1,2])
    # Randomly choose a deformation for each data qubit
    deformation = random.choice(deformation_key, allowed, shape=(code.n_k_d[0],), replace=True)
    basis = ['+X', '-X', '+Z', '-Z'][random.randint(basis_key, (), 0, 4)]
    circ = build_stim_circuit(code, deformation, rounds, basis=basis)
    # circ = noise_model(circ)
    sampler = circ.compile_sampler()
    results = sampler.sample(shots=1)[0]
    # Split the measurements into syndromes and observable
    syndromes = results[:num_syndrome_qubits*rounds].reshape((rounds, num_syndrome_qubits))
    observable = results[num_syndrome_qubits*rounds:].sum() % 2 # Check parity of observable
    # Save the data
    experiments.append({
        "deformation": deformation.tolist(),
        "basis": basis,
        "syndromes": syndromes.tolist(),
    })

Generating data: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:50<00:00,  9.01it/s]


# Save the data

In [51]:
# Helper function to format numbers with scientific prefixes
def scientific_prefix_formatter(num: int) -> str:
    """Formats a number with scientific prefixes for better readability.

    Args:
        num (int): The number to format.

    Returns:
        str: The formatted string with scientific prefix.
    """
    prefixes = {1e9: 'G', 1e6: 'M', 1e3: 'k', 1: '', 1e-3: 'm', 1e-6: 'µ', 1e-9: 'n'}
    for factor, prefix in prefixes.items():
        if num >= factor:
            formatted_num = num / factor
            if formatted_num.is_integer():
                return f"{int(formatted_num)}{prefix}"
    return str(num)

scientific_prefix_formatter(data_points)

'1k'

In [52]:
save_as = f"stim_{noise_model_name}_{code_width}x{code_height}_{scientific_prefix_formatter(data_points)}_r{rounds}"
if os.path.exists(f"../data_sets/{save_as}.json"):
    raise FileExistsError(f"File ../data_sets/{save_as}.json already exists. Please choose a different name to avoid overwriting existing data or manually delete the file.")
with open(f"../data_sets/{save_as}.json", "w") as f:
    json.dump(experiments, f, indent=4)
print(f"Data saved to data_sets/{save_as}.json")

Data saved to data_sets/stim_spin_7x7_1k_r20.json
